In [1]:
#실행 전에 파일 지우기

#telegram_id
mychat_id = "743497117"
mychat_token = '1392629764:AAHbEe3kvUxPmkyKuv-OwCO7JheeWTPX98g'
#빅카인즈 API고유키
bigkinds_access_key = "14fbfb9b-fa97-4f2e-8f6c-093edd4349db"
#네이버 API 고유키
client_id = "AYLJXMlh1XdtYUgIxoAx"
client_secret = "ekLTjHdKry"

from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CommandHandler, CallbackQueryHandler # import modules
import telegram
import urllib.request as ul
import json
import sys
import io
from bs4 import BeautifulSoup
import requests
import logging
import os
import speech_recognition as sr
import playsound
import time
from datetime import datetime
from konlpy.tag import Okt
from gtts import gTTS
from itertools import permutations

date = datetime.today().strftime("%Y-%m-%d")


def today_issue(date, n = 0):
    import requests
    import json
    issue_url = "http://tools.kinds.or.kr:8888/issue_ranking"
    key = bigkinds_access_key
    data = {
        "access_key": key,
        "argument": {
            "date": date,
            "provider": []
        }
    }

    resp = requests.post(issue_url, data=json.dumps(data))
    topics = resp.json()
    topic_list = topics['return_object']['topics'][:5]
    #오늘의 이슈 10개
    topic = [str(idx+1)+"번 " + topic_list[idx]['topic'] for idx, t in enumerate(topic_list)]
    topic_said = ("\n".join(topic))
    #선택이슈 news_id
    news_id  = topic_list[int(n)-1]['news_cluster'][0]
    
    return topic_said, news_id


def bigkinds_content(news_id):
    import requests
    import json
    url = "http://tools.kinds.or.kr:8888/search/news"
    data = {
    "access_key": bigkinds_access_key,
    "argument": {
        "news_ids": [news_id],
        "fields": [
            "content",
            "byline",
            "category",
            "category_incident",
            "images",
            "provider_subject",
            "provider_news_id",
            "publisher_code"]}}
    res = requests.post(url, data=json.dumps(data))
    content_bx = res.json()
    
    if content_bx['return_object']['total_hits'] == 0:
        title = "검색결과가 없습니다"
    else:
        content_bx = content_bx['return_object']['documents']
        title = content_bx[0]['title']
    
    return title

def popular_keywords(date, n=0):
    import requests
    import json
    url_pop = "http://tools.kinds.or.kr:8888/query_rank"
    data = {
       "access_key": bigkinds_access_key,
        "argument": {
            "from": date,
            "until": date,
            "offset": 5,
            "target_access_key": ""
        }
    }
    res = requests.post(url_pop, data=json.dumps(data))
    pop = res.json()
    pop_list = pop['return_object']['queries']
    pkeywords = [str(idx+1)+"번 " + pop_list[idx]["query"] for idx, q in enumerate(pop_list)]
    pkeywords_said = ("\n".join(pkeywords))
    keyword = pop_list[int(n)]['query']
    return pkeywords_said, keyword 

def popular_news(keyword):
    import requests
    import json
    from bs4 import BeautifulSoup
    #client_id = "AYLJXMlh1XdtYUgIxoAx"
    #client_secret = "ekLTjHdKry"
    url = "https://openapi.naver.com/v1/search/news.json"

    header = {
        "X-Naver-Client-Id":client_id,
        "X-Naver-Client-Secret":client_secret
    }

    params = {
        'query':keyword,
        'display':20,
        'start':1000,
        'sort':'sim'

    }

    resp_naver = requests.get(url, params=params, headers=header)
    data_naver = json.loads(resp_naver.text) 
    items = data_naver["items"]
    if(len(items) == 0):
        return None
    
    naver_dec = items[1]["description"].replace("<b>","").replace("</b>","").replace("&quot;","")
    
    
    return naver_dec


def main():
    #안녕하세요 뉴스 챗봇입니다 음성출력
    telgm_token = mychat_token
    bot = telegram.Bot(token = telgm_token)
    
    text='안녕하세요 뉴스 읽어주는 뉴스챗봇입니다:)'   
    bot.sendMessage(chat_id = mychat_id, text = text)
    playsound.playsound('Hi.mp3', True)#픽스
    
    time.sleep(1)
    file_num_string=1
    
    while True:
        text='*****서비스를 선택해주세요*****\n1번: 오늘의 이슈 \n2번 오늘의 인기 검색어\n3번 뉴스 조회\n번호를 선택해주세요.'
        bot.sendMessage(chat_id = mychat_id, text = text)
        
        playsound.playsound('start.mp3', True) #픽스

        # microphone에서 auido source를 생성합니다
        r = sr.Recognizer()
        with sr.Microphone() as source:
            print("Say something!")
            audio = r.listen(source)

        try:
            print("Google Speech Recognition thinks you said : " + r.recognize_google(audio, language='ko'))
        except sr.UnknownValueError:
            print("Google Speech Recognition could not understand audio")
        except sr.RequestError as e:
            print("Could not request results from Google Speech Recognition service; {0}".format(e))
            
        #파일 저장
        with open("microphone-results.wav", "wb") as f:
            f.write(audio.get_wav_data())
            
       

        if(r.recognize_google(audio, language='ko')[0] == '1') or  (r.recognize_google(audio, language='ko')[0] =='일'):
            
            playsound.playsound('issue.mp3', True)#픽스
            
            bot.sendMessage(chat_id = mychat_id, text="*****오늘의 이슈 TOP5*****")
            #date = datetime.today().strftime("%Y-%m-%d")
            
            text = today_issue(date)[0]
            bot.sendMessage(chat_id = mychat_id, text = text)
            myobj = gTTS(text=text, lang = 'ko', slow=False)
            
            file_name = "today_issue" + str(file_num_string) + ".mp3"
            myobj.save(file_name)
            playsound.playsound(file_name, True)
            file_num_string += 1
            
            time.sleep(1)
            playsound.playsound('get_num.mp3', True)#픽스
            time.sleep(2)
            
            r = sr.Recognizer()
            
            while True:
                

                with sr.Microphone() as source:
                    print("Say something!")
                    audio = r.listen(source)

                try:
                    print("Google Speech Recognition thinks you said : " + r.recognize_google(audio, language='ko'))
                except sr.UnknownValueError:
                    print("Google Speech Recognition could not understand audio")
                except sr.RequestError as e:
                    print("Could not request results from Google Speech Recognition service; {0}".format(e))


                if(r.recognize_google(audio, language='ko')[0] == '1' or r.recognize_google(audio, language='ko')[0] == '일'):
                    num = today_issue(date, '1')[1]
                    text = bigkinds_content(num)
                    bot.sendMessage(chat_id = mychat_id, text = text)
                    myobj = gTTS(text=text, lang = 'ko', slow=False)

                    file_name = "selective_today_issue" + str(file_num_string) + ".mp3"
                    file_num_string += 1
                    myobj.save(file_name)
                    playsound.playsound(file_name, True)
                    break

                elif(r.recognize_google(audio, language='ko')[0] == '2' or r.recognize_google(audio, language='ko')[0] == '이'):
                    num = today_issue(date, '2')[1]
                    text = bigkinds_content(num)
                    bot.sendMessage(chat_id = mychat_id, text = text)
                    myobj = gTTS(text=text, lang = 'ko', slow=False)
                    file_name = "selective_today_issue" + str(file_num_string) + ".mp3"
                    file_num_string += 1
                    myobj.save(file_name)
                    playsound.playsound(file_name, True)
                    break
                    
                elif(r.recognize_google(audio, language='ko')[0] == '3' or r.recognize_google(audio, language='ko')[0] == '삼'):
                    num = today_issue(date, '3')[1]
                    text = bigkinds_content(num)
                    bot.sendMessage(chat_id = mychat_id, text = text)
                    myobj = gTTS(text=text, lang = 'ko', slow=False)
                    file_name = "selective_today_issue" + str(file_num_string) + ".mp3"
                    file_num_string += 1
                    myobj.save(file_name)
                    playsound.playsound(file_name, True)
                    break
                    
                elif(r.recognize_google(audio, language='ko')[0] == '4' or r.recognize_google(audio, language='ko')[0] == '사'):
                    num = today_issue(date, '4')[1]
                    text = bigkinds_content(num)
                    bot.sendMessage(chat_id = mychat_id, text = text)
                    myobj = gTTS(text=text, lang = 'ko', slow=False)
                    file_name = "selective_today_issue" + str(file_num_string) + ".mp3"
                    file_num_string += 1
                    myobj.save(file_name)
                    playsound.playsound(file_name, True)
                    break
                    
                elif(r.recognize_google(audio, language='ko')[0] == '5' or r.recognize_google(audio, language='ko')[0] == '오'):
                    num = today_issue(date, '5')[1]
                    text = bigkinds_content(num)
                    bot.sendMessage(chat_id = mychat_id, text = text)
                    myobj = gTTS(text=text, lang = 'ko', slow=False)
                    file_name = "selective_today_issue" + str(file_num_string) + ".mp3"
                    file_num_string += 1
                    myobj.save(file_name)
                    playsound.playsound(file_name, True)
                    break

                else:
                    playsound.playsound('re_num.mp3', True)
                    continue
                
 #############뉴스 조회 / 음성 입력 받아서 짜르고 -> 전체 단어들에 대한 조합을 통해서 search ###########    
        elif(r.recognize_google(audio, language='ko')[0] == '2') or (r.recognize_google(audio, language='ko')[0] =='이'):
            playsound.playsound('search.mp3', True)
            time.sleep(1)
            text=popular_keywords(date)[0]
            bot.sendMessage(chat_id = mychat_id, text="*****인기 검색어 TOP5*****")
            bot.sendMessage(chat_id = mychat_id, text=text) 
            
            myobj = gTTS(text=text, lang = 'ko', slow=False)
            file_name = "today_ranking" + str(file_num_string) + ".mp3"
            myobj.save(file_name)
            playsound.playsound(file_name, True)
            file_num_string += 1
            time.sleep(2)
            playsound.playsound('get_num.mp3', True)#픽스
            time.sleep(2)
            
            while True:
                with sr.Microphone() as source:
                    print("Say something!")
                    audio = r.listen(source)
                    
                try:
                    print("Google Speech Recognition thinks you said : " + r.recognize_google(audio, language='ko'))
                except sr.UnknownValueError:
                    print("Google Speech Recognition could not understand audio")
                except sr.RequestError as e:
                    print("Could not request results from Google Speech Recognition service; {0}".format(e))

                if(r.recognize_google(audio, language='ko')[0] == '1' or r.recognize_google(audio, language='ko')[0] == '일'):
                        text = popular_news(popular_keywords(date, 0)[1])
                        bot.sendMessage(chat_id = mychat_id, text = text)
                        myobj = gTTS(text=text, lang = 'ko', slow=False)

                        file_name = "selective_today_ranking" + str(file_num_string) + ".mp3"
                        file_num_string += 1
                        myobj.save(file_name)
                        playsound.playsound(file_name, True)
                        break

                elif(r.recognize_google(audio, language='ko')[0] == '2' or r.recognize_google(audio, language='ko')[0] == '이'):
                    text = popular_news(popular_keywords(date, 1)[1])
                    bot.sendMessage(chat_id = mychat_id, text = text)
                    myobj = gTTS(text=text, lang = 'ko', slow=False)

                    file_name = "selective_today_ranking" + str(file_num_string) + ".mp3"
                    file_num_string += 1
                    myobj.save(file_name)
                    playsound.playsound(file_name, True)
                    break

                elif(r.recognize_google(audio, language='ko')[0] == '3' or r.recognize_google(audio, language='ko')[0] == '삼'):
                    text = popular_news(popular_keywords(date, 2)[1])
                    bot.sendMessage(chat_id = mychat_id, text = text)
                    myobj = gTTS(text=text, lang = 'ko', slow=False)

                    file_name = "selective_today_ranking" + str(file_num_string) + ".mp3"
                    file_num_string += 1
                    myobj.save(file_name)
                    playsound.playsound(file_name, True)
                    break

                elif(r.recognize_google(audio, language='ko')[0] == '4' or r.recognize_google(audio, language='ko')[0] == '사'):
                    text = popular_news(popular_keywords(date, 3)[1])
                    bot.sendMessage(chat_id = mychat_id, text = text)
                    myobj = gTTS(text=text, lang = 'ko', slow=False)

                    file_name = "selective_today_ranking" + str(file_num_string) + ".mp3"
                    file_num_string += 1
                    myobj.save(file_name)
                    playsound.playsound(file_name, True)
                    break

                elif(r.recognize_google(audio, language='ko')[0] == '5' or r.recognize_google(audio, language='ko')[0] == '오'):
                    text = popular_news(popular_keywords(date, 4)[1])
                    bot.sendMessage(chat_id = mychat_id, text = text)
                    myobj = gTTS(text=text, lang = 'ko', slow=False)

                    file_name = "selective_today_ranking" + str(file_num_string) + ".mp3"
                    file_num_string += 1
                    myobj.save(file_name)
                    playsound.playsound(file_name, True)
                    break

                else:
                    playsound.playsound('re_num.mp3', True)
                    continue

                #############쥬피터노트북 내에서 microphone으로 다시 음성을 입력받는다.#############
                with sr.Microphone() as source:
                        print("Say something!")
                        audio = r.listen(source)

                try:
                    print("Google Speech Recognition thinks you said : " + r.recognize_google(audio, language='ko'))
                except sr.UnknownValueError:
                    print("Google Speech Recognition could not understand audio")
                except sr.RequestError as e:
                    print("Could not request results from Google Speech Recognition service; {0}".format(e))

                if(r.recognize_google(audio, language='ko')[0] == '1' or r.recognize_google(audio, language='ko')[0] == '일'):
                        text = popular_news(popular_keywords(date, 0)[1])
                        bot.sendMessage(chat_id = mychat_id, text = text)
                        myobj = gTTS(text=text, lang = 'ko', slow=False)

                        file_name = "selective_today_ranking" + str(file_num_string) + ".mp3"
                        file_num_string += 1
                        myobj.save(file_name)
                        playsound.playsound(file_name, True)
                        break

                elif(r.recognize_google(audio, language='ko')[0] == '2' or r.recognize_google(audio, language='ko')[0] == '이'):
                    text = popular_news(popular_keywords(date, 1)[1])
                    bot.sendMessage(chat_id = mychat_id, text = text)
                    myobj = gTTS(text=text, lang = 'ko', slow=False)

                    file_name = "selective_today_ranking" + str(file_num_string) + ".mp3"
                    file_num_string += 1
                    myobj.save(file_name)
                    playsound.playsound(file_name, True)
                    break

                elif(r.recognize_google(audio, language='ko')[0] == '3' or r.recognize_google(audio, language='ko')[0] == '삼'):
                    text = popular_news(popular_keywords(date, 2)[1])
                    bot.sendMessage(chat_id = mychat_id, text = text)
                    myobj = gTTS(text=text, lang = 'ko', slow=False)

                    file_name = "selective_today_ranking" + str(file_num_string) + ".mp3"
                    file_num_string += 1
                    myobj.save(file_name)
                    playsound.playsound(file_name, True)
                    break

                elif(r.recognize_google(audio, language='ko')[0] == '4' or r.recognize_google(audio, language='ko')[0] == '사'):
                    text = popular_news(popular_keywords(date, 3)[1])
                    bot.sendMessage(chat_id = mychat_id, text = text)
                    myobj = gTTS(text=text, lang = 'ko', slow=False)

                    file_name = "selective_today_ranking" + str(file_num_string) + ".mp3"
                    file_num_string += 1
                    myobj.save(file_name)
                    playsound.playsound(file_name, True)
                    break

                elif(r.recognize_google(audio, language='ko')[0] == '5' or r.recognize_google(audio, language='ko')[0] == '오'):
                    text = popular_news(popular_keywords(date, 4)[1])
                    bot.sendMessage(chat_id = mychat_id, text = text)
                    myobj = gTTS(text=text, lang = 'ko', slow=False)

                    file_name = "selective_today_ranking" + str(file_num_string) + ".mp3"
                    file_num_string += 1
                    myobj.save(file_name)
                    playsound.playsound(file_name, True)
                    break

                else:
                    playsound.playsound('re_num.mp3', True)
                    continue
                
                
            
#######################################
        elif(r.recognize_google(audio, language='ko')[0] == '3') or (r.recognize_google(audio, language='ko')[0] =='삼'):
            bot.sendMessage(chat_id = mychat_id, text="*****알고 싶은 뉴스가 무엇인지 알려주세요.*****")
            
            playsound.playsound('get_search.mp3', True)#픽스
            time.sleep(2)
            

            #############쥬피터 노트북에서 음성을 입력받음#############
            with sr.Microphone() as source:
                    print("Say something!")
                    audio = r.listen(source)       
            try:
                print("Google Speech Recognition thinks you said : " + r.recognize_google(audio, language='ko'))
            except sr.UnknownValueError:
                print("Google Speech Recognition could not understand audio")
            except sr.RequestError as e:
                print("Could not request results from Google Speech Recognition service; {0}".format(e))
                
            ########형태소 분리, 뭐가 좋을지 몰라서 phrases 사용###########
            okt = Okt()
            phrase_list = okt.nouns(r.recognize_google(audio, language='ko'))
            permutation_list = list(permutations(phrase_list, 2)) # 조합 함수를 사용해서 모든 단어 조합을 추출
            key_word_check = 0 ## 기사를 탐색했는지 확인하려고 변수 선언
            okt = Okt()
            phrase_list = okt.nouns(r.recognize_google(audio, language='ko'))
            permutation_list = list(permutations(phrase_list, 2)) # 조합 함수를 사용해서 모든 단어 조합을 추출
            key_word_check = 0 ## 기사를 탐색했는지 확인하려고 변수 선언
            for i in permutation_list:
                keyword = i[0] + " " + i[1] ## search할 때, keyword는 스페이스바를 포함해서 들어감 ex)'추미애 검찰'
                ### NULL이 아닌경우 keyword_check = 1로 수정 후 탈출 ###
                
                return_keyword = popular_news(keyword)
                
                if(return_keyword == None):
                    continue
                ######### NULL일 경우 조합이 끝날때 까지 탐색 #########
                else:
                    key_word_check = 1
                    break
            #이때는 검색한 기사가 존재함 기사 출력 
            bot.sendMessage(chat_id = mychat_id, text= '검색하신 뉴스 결과를 알려드리겠습니다.')
            playsound.playsound('yes_content.mp3', True)#픽스
            time.sleep(1)

            if(key_word_check == 1):
                bot.sendMessage(chat_id = mychat_id, text=return_keyword)
                myobj = gTTS(text=return_keyword, lang = 'ko', slow=False)
                file_name = "search_file" + str(file_num_string) + ".mp3"
                file_num_string += 1
                myobj.save(file_name)
                playsound.playsound(file_name, True)
                time.sleep(2)
                #기사출력
            else:
                bot.sendMessage(chat_id = mychat_id, text= '해당 내용의 뉴스가 존재하지 않습니다:(')
                ##이 부분은 mp3파일을 미리 만들어서 실행 or 아니면 gTTS 사용해서 text로 저장해서 사용
                playsound.playsound('no_content.mp3', True)#픽스
                time.sleep(2)

        elif(r.recognize_google(audio, language='ko')[0] == '4') or (r.recognize_google(audio, language='ko')[0] == '사'):
        
                bot.sendMessage(chat_id = mychat_id, text="----종료----")
                ###############종료 mp3 코드 추가##############
                break
        else:
            bot.sendMessage(chat_id = mychat_id, text="----잘못된 명령어 or 잘못된 번호----")
            playsound.playsound('re_num.mp3', True)
            time.sleep(2)
        time.sleep(2)

In [2]:
main()

Say something!
Google Speech Recognition thinks you said : 3번
Say something!
Google Speech Recognition thinks you said : 추미애 아들에 대해 알려 줘
Say something!
Google Speech Recognition could not understand audio


UnknownValueError: 